In [ ]:
!pip install transformers accelerate bitsandbytes sentence-transformers faiss-cpu bitsandbytes

## Logging in to hugging face

In [ ]:
from huggingface_hub import login

login(input("Enter your hugging face token: "))

Enter your hugging face token: 


## Llama initialization

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config
)
llm = pipeline("text-generation", model=model, tokenizer=tokenizer)
tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [ ]:
inputs = tokenizer("What is RAG system ?", return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


What is RAG system? RAG system is a method used to categorize the risk associated with a project, process or activity. The term RAG stands for Red, Amber, Green, which are the three colors used to indicate the level of risk.
What is RAG


## Assistant prompts

In [ ]:
import yaml

try:
  from google.colab import files
  print("please upload docs")
  uploaded = files.upload()
except:
  print("not running on colab")

with open("assistant-prompts.yml", "r") as f:   # change this to the correct path, this path is meant to work in colab
    PROMPTS = yaml.safe_load(f)

def clean_doc_text(doc):
    """
    Remove Markdown headers (#, ##, ###, etc.) and extra whitespace from a document.
    """

    doc = re.sub(r'^#{1,6}\s*', '', doc, flags=re.MULTILINE)

    doc = doc.strip()
    return doc

def build_prompt(prompt_type, context, question, retrieved_docs=None):
    cfg = PROMPTS[prompt_type]

    if retrieved_docs and len(retrieved_docs) > 0:
        cleaned_docs = [clean_doc_text(doc) for doc in retrieved_docs]
        sources_text = "\n\n".join(cleaned_docs)
        explanation_text = f"This answer is based on {len(retrieved_docs)} document(s) from the knowledge base."
    else:
        sources_text = "None"
        explanation_text = "No relevant documents found."

    prompt = f"""{cfg['role']}

Goal:
{cfg['goal']}

Context:
{context}

Question: {question}

{cfg['response_format']}
{explanation_text}
"""
    return prompt



please upload docs


Saving assistant-prompts.yml to assistant-prompts (1).yml


## FAISS

In [ ]:
from sentence_transformers import SentenceTransformer, util
import faiss
import numpy as np

embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

try:
  from google.colab import files
  print("please upload docs")
  uploaded = files.upload()
except:
  print("not running on colab")

with open("knowledge-base.md", "r") as f:
    kb_text = f.read()

raw_docs = kb_text.split("---")
documents = [doc.strip() for doc in raw_docs if doc.strip()]

embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = embedder.encode(documents, convert_to_numpy=True)
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

please upload docs


Saving knowledge-base.md to knowledge-base (1).md


## rag query

In [ ]:
import re

def rag_query(query, threshold=2, k=3):
    qb = embedder.encode([query], convert_to_numpy=True)
    distances, indices = index.search(qb, k)

    retrieved_docs = [documents[i] for i in indices[0] if i < len(documents)]
    min_distance = float(distances[0][0]) if len(distances[0]) > 0 else 1.0

    if len(retrieved_docs) == 0 or min_distance > threshold:
        prompt_type = "no_context_prompt"
        context = ""
    elif len(retrieved_docs) == 1:
        prompt_type = "base_retrieval_prompt"
        context = retrieved_docs[0]
    else:
        prompt_type = "multi_doc_prompt"
        context = "\n\n".join(retrieved_docs)

    prompt = build_prompt(prompt_type, context, query, retrieved_docs=retrieved_docs)
    response = llm(prompt, max_new_tokens=512, temperature=0.2, do_sample=False)
    full_text = response[0]['generated_text'].strip()

    if "Answer:" in full_text:
        return full_text.split("Answer:", 1)[-1].strip()
    else:
        return full_text

rag_query("what is the refund time ?")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'This answer is based on 3 document(s) from the knowledge base.\nAccording to Document 7: Returns and Refunds, refunds are processed within 7–10 business days after items are inspected. This is the standard timeframe for refunds. However, please note that this timeframe may vary depending on the specific circumstances of the return. For more information, please refer to Document 7.'

## Evaluating q&a
### using semantic evaluation



In [ ]:
import re


qa_file = "ground-truth-qa.md"
with open(qa_file, "r") as f:
    qa_text = f.read()

qa_pairs = qa_text.split("---")
qa_pairs = [pair.strip() for pair in qa_pairs if pair.strip()]
print("QA: ", len(qa_pairs)-1)

def clean_text(text):
    text = re.sub(r"📖.*", "", text)
    text = re.sub(r"`+", "", text)
    text = text.replace("_____", "")
    return text.strip()

def evaluate_rag(k=3, threshold=0.7):
    total = len(qa_pairs) - 1
    correct = 0
    faithful = 0
    results = []

    for i, pair in enumerate(qa_pairs[1:], start=1):
        try:
            question, expected = pair.split('**Answer:**')
        except ValueError:
            print(f"Skipping malformed QA pair: {pair}")
            continue

        expected_answer = clean_text(expected)
        raw_answer = rag_query(question, k=k)
        generated_answer = clean_text(raw_answer)

        # semantic similarity
        sim = util.cos_sim(
            embedder.encode(generated_answer, convert_to_tensor=True),
            embedder.encode(expected_answer, convert_to_tensor=True)
        ).item()

        is_correct = sim >= threshold
        is_faithful = "not in the provided documents" not in generated_answer.lower()

        if is_correct:
            correct += 1
        if is_faithful:
            faithful += 1

        results.append({
            "Question": question.strip(),
            "Expected": expected_answer,
            "Answer": generated_answer,
            "Similarity": sim,
            "Correct": is_correct,
            "Faithful": is_faithful
        })

        print(f"[Q{i}] {question.strip()}")
        print(f"Expected: {expected_answer}")
        print(f"Got: {generated_answer.strip()}")
        print(f"Similarity: {sim:.2f} | Correct: {is_correct}\n")

    accuracy = (correct / total) * 100
    faithfulness_score = (faithful / total) * 100
    print("=== Final Evaluation ===")
    print(f"Accuracy: {accuracy:.1f}% ({correct}/{total})")
    print(f"Faithfulness: {faithfulness_score:.1f}% ({faithful}/{total})")

    return results, accuracy, faithfulness_score

results, accuracy, faithfulness = evaluate_rag(k=3, threshold=0.6)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


QA:  20


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[Q1] ### Q1: How do I create an account on Shoplite?
Expected: You can create an account using your email and password, or sign up with Google/Facebook login. A verified email is required for confirmations and password resets.
Got: This answer is based on 3 document(s) from the knowledge base.
To create an account on Shoplite, you need to sign up with a valid email address, a password, and acceptance of Shoplite's terms of service. You can also register using Google or Facebook login. After creating an account, you can manage profile details, shipping addresses, and payment methods from the Account Settings page. A verified email is required for order confirmations and password resets. Accounts that remain inactive for long periods may be flagged for review, but they are not automatically deleted. Shoplite ensures user data is stored securely, following encryption standards and privacy rules.
Similarity: 0.60 | Correct: True



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

[Q17] ### Q17: How secure is my personal data on Shoplite?
Expected: Shoplite complies with GDPR, encrypts all data transfers, and lets users delete their accounts anytime. Payment details are never stored directly.
Got: This answer is based on 3 document(s) from the knowledge base.
Shoplite takes privacy seriously and complies with GDPR and other privacy laws. Personal data such as name, email, and address is collected for order fulfillment and customer service. Payment details are never stored directly. Shoplite uses encryption for all data transfers and performs regular audits of its systems to prevent breaches. Additionally, user data is stored securely, following encryption standards and privacy rules. Customers can request account deletion at any time, which removes personal data except for records required by law. Cookies are used to improve browsing, and customers can adjust cookie settings. Overall, Shoplite ensures the security and privacy of your personal data.
Similarity: 0

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[Q18] ### Q18: Does Shoplite offer student discounts?
Expected: Yes. Student discounts are sometimes offered through partner verification services.
Got: This answer is based on 3 document(s) from the knowledge base.
Yes, Shoplite sometimes offers student discounts, which require verification through partner services. (Document 13: Promotions and Discounts) These discounts are not mentioned in the loyalty program (Document 12), but they are mentioned in the Shoplite Overview (Document 1) as a type of promotion. The loyalty program does not provide information on student discounts. The Shoplite Overview mentions promotions, which include student discounts, but does not provide details on the loyalty program. The answer is accurate, concise, and brief.
Similarity: 0.76 | Correct: True



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[Q19] ### Q19: How does Shoplite contribute to the environment?
Expected: It uses recyclable packaging, eco-friendly couriers where possible, and offers “green shipping” as a slower but sustainable delivery option.
Got: This answer is based on 3 document(s) from the knowledge base.
Shoplite contributes to the environment by making sustainable choices. Packaging is recyclable, and customers are encouraged to recycle. The company partners with eco-friendly couriers where possible. Additionally, customers can opt for “green shipping,” which uses slower but lower-carbon delivery methods. Shoplite also supports charity campaigns, donating a portion of sales during special events to environmental causes. By making these choices, Shoplite aims to balance convenience with responsibility.
Similarity: 0.68 | Correct: True

[Q20] ### Q20: What should I do if the Shoplite app crashes?
Expected: First try clearing cache, updating the app, or restarting your device. If it persists, contact Shoplite 

# Ngrok and flask

In [ ]:
!pip install flask pyngrok

In [ ]:
import json
from flask import Flask, request, jsonify
from pyngrok import ngrok

app = Flask(__name__)

# /health route
@app.route("/health", methods=["GET"])
def health():
    return jsonify({"status": "ok", "message": "RAG system is running"})

# /ping route
@app.route("/ping", methods=["POST"])
def ping():
    data = request.get_json()
    query = data.get("query", "")
    if not query:
        return jsonify({"error": "No query provided"}), 400

    inputs = tokenizer(query, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=128)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return jsonify({"query": query, "response": answer})

# /chat route
@app.route("/chat", methods=["POST"])
def chat():
    data = request.get_json()
    query = data.get("query", "")
    if not query:
        return jsonify({"error": "No query provided"}), 400

    answer = rag_query(query)

    return jsonify({
        "query": query,
        "answer": answer
    })

ngrok_token = input("Enter your ngrok token: ")
ngrok.set_auth_token(ngrok_token)
public_url = ngrok.connect(5000).public_url
print(" * Ngrok tunnel open:", public_url)

app.run(port=5000)


Enter your ngrok token: 
 * Ngrok tunnel open: https://nondiaphanous-unonerously-mimi.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
